In [1]:
from qiskit.transpiler.preset_passmanagers import generate_preset_pass_manager
from qiskit.visualization import *
# Importing standard Qiskit libraries and configuring account
from qiskit_ibm_runtime import QiskitRuntimeService, Options
#import qiskit_ibm_runtime 
from qiskit.primitives import Sampler as Sampler_2
#from qiskit.providers.fake_provider import FakeMumbaiV2
from qiskit_aer.primitives import Sampler
#from qiskit.utils import algorithm_globals
#Load feature maps
from qiskit_aer import AerSimulator
from qiskit_aer.noise import NoiseModel
from qiskit.circuit.library import ZZFeatureMap
from qiskit_algorithms.state_fidelities import ComputeUncompute
#from Kernels.src.ComputeUncompute import ComputeUncompute
from Kernels.src.ComputeUncompute_2 import ComputeUncompute_2
from qiskit_machine_learning.kernels import FidelityQuantumKernel

import numpy as np

from sklearn.preprocessing import MinMaxScaler

In [2]:
########################QUANTUM SESSION#########################################################
# Loading your IBM Quantum account(s)
print('Loading IBM Quantum account')
# Loading your IBM Quantum account(s)
service=QiskitRuntimeService(channel="ibm_quantum",token="95e70510a429e03c7cc4ce4c5c979c794135dbacffbebe5325dcd8b627cfe42c9f61e91945ba1adea992f81849c74cc5e0ed1ab93977db5084dd26a28fd25dd8")
print('selecting backend')
#backend = service.get_backend('ibmq_qasm_simulator')
backend_or=service.least_busy(operational=True, simulator=False)
#backend=FakeMumbaiV2()
noise_model = NoiseModel.from_backend(backend_or)
backend=AerSimulator.from_backend(backend_or)

print(backend)
target = backend.target
coupling_map = target.build_coupling_map()

Loading IBM Quantum account
selecting backend
AerSimulator(''aer_simulator(ibm_cairo)'
             noise_model=<NoiseModel on ['measure', 'x', 'reset', 'sx', 'cx', 'id']>)


In [3]:
print('FT map instance')
#Instance FTMAP
n_qubits=4
reps=1
ft_map = ZZFeatureMap(feature_dimension=n_qubits, reps=1)

#transpile circuit
print('transpile circuit')
pm=generate_preset_pass_manager(backend=backend,optimization_level=1)
print(type(pm))
ft_map_t_qs = pm.run(ft_map)

FT map instance
transpile circuit


<class 'qiskit.transpiler.passmanager.StagedPassManager'>


In [4]:
#Generate sample data
X_train=np.random.rand(10,4)

scaler = MinMaxScaler(feature_range=(0, 1*np.pi))
scaler.fit(X_train)
X_train_scaled = scaler.transform(X_train)

In [6]:

# Create a quantum kernel based on the transpiled feature map
#Set Primitive sampler
sampler = Sampler(backend_options={"noise_model": noise_model, "shots": 2000, "seed_simulator":42})

sampler_2 = Sampler_2(options={"noise_model": noise_model, "shots": 2000, "seed_simulator":42})

### Run as fidelity kernel

In [8]:
print('Run as fidelity kernel normal way')
fidelity = ComputeUncompute(sampler=sampler)
fidelity_2=ComputeUncompute(sampler=sampler_2)


Run as fidelity kernel normal way


In [9]:
qk=FidelityQuantumKernel(feature_map=ft_map_t_qs,fidelity=fidelity_2)
qk.evaluate(X_train_scaled)

KeyboardInterrupt: 

In [ ]:
qk=FidelityQuantumKernel(feature_map=ft_map_t_qs,fidelity=fidelity)
qk.evaluate(X_train_scaled)